# Comprehensive Statistical Tables for Supplementary Material

This notebook generates comprehensive statistical tables for all features across all modalities.

For each feature set (Performance, ECG, GSR, Eye-Tracking, Pose), the tables include:
- Feature name
- Mean ± SD for each condition (Low, Moderate, High)
- Overall condition effect (F-statistic, p-value)
- Pairwise comparisons (L vs M, L vs H, M vs H) with p-values

Outputs:
- CSV files for each feature set
- LaTeX table code for copy-paste into Overleaf

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys

# Import stats utilities
from stats_figures import run_rpy2_lmer

Error importing in API mode: ImportError('On Windows, cffi mode "ANY" is only "ABI".')
Trying to import in ABI mode.


## Configuration

In [2]:
# Base directory (parent of stats_utils)
BASE_DIR = Path("..").resolve()

# Data paths
PERFORMANCE_DATA = BASE_DIR / "MATB_performance" / "data" / "processed" / "combined" / "performance_metrics_all.csv"
EYETRACKING_DATA = BASE_DIR / "eye_tracking" / "data" / "processed" / "combined" / "eyegaze_metrics_all.csv"
GSR_DATA = BASE_DIR / "gsr" / "data" / "processed" / "combined" / "gsr_features_all.csv"
ECG_DATA = BASE_DIR / "ecg" / "data" / "processed" / "combined" / "ecg_features_all.csv"

# Pose data
POSE_METHOD = "procrustes_global"
POSE_DATA = BASE_DIR / "Pose" / "data" / "processed" / "linear_metrics" / f"{POSE_METHOD}_linear.csv"

# Output directory
OUTPUT_DIR = Path(".").resolve()
OUTPUT_DIR.mkdir(exist_ok=True)

print("✓ Configuration complete")

✓ Configuration complete


## Load Data

In [3]:
def prepare_dataframe(df, participant_col="participant"):
    """
    Prepare dataframe for statistical analysis.
    """
    df = df.copy()
    
    # Rename participant column if needed
    if participant_col in df.columns and participant_col != "participant_id":
        df = df.rename(columns={participant_col: "participant_id"})
    
    # Ensure condition is uppercase
    if "condition" in df.columns:
        df["condition"] = df["condition"].astype(str).str.strip().str.upper()
    
    # Add session_order_numeric if not present
    if "session_order_numeric" not in df.columns:
        df["session_order_numeric"] = 1
    
    return df


print("Loading datasets...")
performance_df = prepare_dataframe(pd.read_csv(PERFORMANCE_DATA))
print(f"  ✓ Performance: {performance_df.shape[0]} rows")

eyetracking_df = prepare_dataframe(pd.read_csv(EYETRACKING_DATA))
print(f"  ✓ Eye-tracking: {eyetracking_df.shape[0]} rows")

gsr_df = prepare_dataframe(pd.read_csv(GSR_DATA))
print(f"  ✓ GSR: {gsr_df.shape[0]} rows")

ecg_df = prepare_dataframe(pd.read_csv(ECG_DATA))
print(f"  ✓ ECG: {ecg_df.shape[0]} rows")

pose_df = prepare_dataframe(pd.read_csv(POSE_DATA))
print(f"  ✓ Pose: {pose_df.shape[0]} rows")

print("\n✓ All data loaded")

Loading datasets...
  ✓ Performance: 1935 rows
  ✓ Eye-tracking: 1875 rows
  ✓ GSR: 1782 rows
  ✓ ECG: 1680 rows
  ✓ Pose: 2021 rows

✓ All data loaded


## Define Feature Sets

Select which features to analyze for each modality.

In [4]:
# Get all feature columns (excluding metadata)
excluded_cols = ['participant_id', 'participant', 'condition', 'window_index', 
                'start_time', 'end_time', 'session_order_numeric', 't_start_sec', 
                't_end_sec', 'filename']

# Performance features
performance_features = [c for c in performance_df.columns if c not in excluded_cols]
print(f"Performance features ({len(performance_features)}): {performance_features}")

# Eye-tracking features
eyetracking_features = [c for c in eyetracking_df.columns if c not in excluded_cols]
print(f"\nEye-tracking features ({len(eyetracking_features)}): {eyetracking_features}")

# GSR features
gsr_features = [c for c in gsr_df.columns if c not in excluded_cols]
print(f"\nGSR features ({len(gsr_features)}): {gsr_features}")

# ECG features (limit to HRV features for manageability)
ecg_features = [c for c in ecg_df.columns if c.startswith('HRV') and c not in excluded_cols]
print(f"\nECG features ({len(ecg_features)}): {ecg_features[:20]}... (showing first 20)")

# Pose features (select a subset of meaningful features)
pose_features = [c for c in pose_df.columns 
                if ('mean' in c.lower() or 'rms' in c.lower() or 'std' in c.lower()) 
                and c not in excluded_cols]
print(f"\nPose features ({len(pose_features)}): {pose_features[:20]}... (showing first 20)")

Performance features (10): ['sysmon_failure_rate', 'sysmon_average_reaction_times', 'comms_failure_rate', 'comms_events', 'comms_own_events', 'comms_average_reaction_times', 'track_failure_rate', 'resman_failure_rate', 'average_accuracy', 'average_reaction_time']

Eye-tracking features (14): ['pct_missing', 'mean_vel', 'max_vel', 'mean_acc', 'rms_disp', 'fix_count', 'fix_mean_dur', 'fix_rate', 'blink_count', 'blink_mean_dur', 'blink_rate', 'sac_count', 'sac_mean_dur', 'sac_rate']

GSR features (4): ['SCR_Peaks_N', 'SCR_Peaks_Amplitude_Mean', 'EDA_Tonic_SD', 'EDA_Autocorrelation']

ECG features (83): ['HRV_MeanNN', 'HRV_SDNN', 'HRV_RMSSD', 'HRV_SDSD', 'HRV_CVNN', 'HRV_CVSD', 'HRV_MedianNN', 'HRV_MadNN', 'HRV_MCVNN', 'HRV_IQRNN', 'HRV_SDRMSSD', 'HRV_Prc20NN', 'HRV_Prc80NN', 'HRV_pNN50', 'HRV_pNN20', 'HRV_MinNN', 'HRV_MaxNN', 'HRV_HTI', 'HRV_TINN', 'HRV_LF']... (showing first 20)

Pose features (99): ['head_rotation_rad_rms', 'head_rotation_rad_mean', 'head_rotation_rad_std', 'head_tx_rms

## Statistical Analysis Functions

In [5]:
def analyze_all_features(df, features, feature_set_name):
    """
    Run statistical analysis for all features in a dataset.
    
    Returns:
        DataFrame with columns: Feature, Low_Mean, Low_SD, Mod_Mean, Mod_SD, 
                               High_Mean, High_SD, p_L_vs_M, p_L_vs_H, p_M_vs_H
    """
    results_list = []
    
    print(f"\n{'='*60}")
    print(f"Analyzing {feature_set_name} features")
    print(f"{'='*60}")
    
    for feature in features:
        print(f"\n  Processing: {feature}")
        
        try:
            # Run mixed-effects model
            pvals, means, cis = run_rpy2_lmer(df, feature, feature)
            
            # Calculate SD from CI (CI = mean ± 1.96*SE, so SE = CI_width/(2*1.96))
            # SD = SE * sqrt(n), but we'll use the full data SD instead
            condition_groups = df.groupby('condition')[feature]
            sds = condition_groups.std()
            
            # Create result row
            result = {
                'Feature': feature,
                'Low_Mean': means.get('L', np.nan),
                'Low_SD': sds.get('L', np.nan),
                'Mod_Mean': means.get('M', np.nan),
                'Mod_SD': sds.get('M', np.nan),
                'High_Mean': means.get('H', np.nan),
                'High_SD': sds.get('H', np.nan),
                'p_L_vs_M': pvals.get(('L', 'M'), np.nan),
                'p_L_vs_H': pvals.get(('L', 'H'), np.nan),
                'p_M_vs_H': pvals.get(('M', 'H'), np.nan)
            }
            
            results_list.append(result)
            print(f"    ✓ Complete")
            
        except Exception as e:
            print(f"    ✗ Error: {e}")
            # Add row with NaNs
            result = {
                'Feature': feature,
                'Low_Mean': np.nan,
                'Low_SD': np.nan,
                'Mod_Mean': np.nan,
                'Mod_SD': np.nan,
                'High_Mean': np.nan,
                'High_SD': np.nan,
                'p_L_vs_M': np.nan,
                'p_L_vs_H': np.nan,
                'p_M_vs_H': np.nan
            }
            results_list.append(result)
    
    results_df = pd.DataFrame(results_list)
    print(f"\n✓ Completed {feature_set_name} analysis")
    
    return results_df

## Table Formatting Functions

In [6]:
def format_pvalue(p):
    """Format p-value for display."""
    if pd.isna(p):
        return '-'
    elif p < 0.001:
        return '<0.001***'
    elif p < 0.01:
        return f'{p:.3f}**'
    elif p < 0.05:
        return f'{p:.3f}*'
    else:
        return f'{p:.3f}'


def create_latex_longtable(df, caption, label, feature_set_name):
    """
    Create LaTeX longtable code from results dataframe.
    Uses longtable for tables that span multiple pages.
    
    Table fits full page width with columns:
    Feature | Low | Moderate | High | L vs M | L vs H | M vs H
    """
    latex = []
    
    # Table header
    latex.append("\\begin{longtable}{l c c c c c c}")
    latex.append("\\caption{" + caption + "} \\label{" + label + "} \\\\")
    latex.append("\\hline")
    latex.append("Feature & Low & Moderate & High & \\textit{p} L vs M & \\textit{p} L vs H & \\textit{p} M vs H \\\\")
    latex.append("\\hline")
    latex.append("\\endfirsthead")
    latex.append("")
    latex.append("\\multicolumn{7}{c}")
    latex.append("{\\tablename\\ \\thetable\\ -- \\textit{Continued from previous page}} \\\\")
    latex.append("\\hline")
    latex.append("Feature & Low & Moderate & High & \\textit{p} L vs M & \\textit{p} L vs H & \\textit{p} M vs H \\\\")
    latex.append("\\hline")
    latex.append("\\endhead")
    latex.append("")
    latex.append("\\hline \\multicolumn{7}{r}{\\textit{Continued on next page}} \\\\")
    latex.append("\\endfoot")
    latex.append("")
    latex.append("\\hline")
    latex.append("\\multicolumn{7}{l}{\\textit{Note: Values shown as Mean $\\pm$ SD. Significance: * $p < 0.05$, ** $p < 0.01$, *** $p < 0.001$.}} \\\\")
    latex.append("\\endlastfoot")
    latex.append("")
    
    # Table rows
    for _, row in df.iterrows():
        feature_name = row['Feature'].replace('_', '\\_')  # Escape underscores
        
        # Format mean ± SD
        low_str = f"{row['Low_Mean']:.2f} $\\pm$ {row['Low_SD']:.2f}" if not pd.isna(row['Low_Mean']) else '-'
        mod_str = f"{row['Mod_Mean']:.2f} $\\pm$ {row['Mod_SD']:.2f}" if not pd.isna(row['Mod_Mean']) else '-'
        high_str = f"{row['High_Mean']:.2f} $\\pm$ {row['High_SD']:.2f}" if not pd.isna(row['High_Mean']) else '-'
        
        # Format p-values
        p_lm = format_pvalue(row['p_L_vs_M'])
        p_lh = format_pvalue(row['p_L_vs_H'])
        p_mh = format_pvalue(row['p_M_vs_H'])
        
        latex.append(f"{feature_name} & {low_str} & {mod_str} & {high_str} & {p_lm} & {p_lh} & {p_mh} \\\\")
    
    # Table footer
    latex.append("\\end{longtable}")
    
    return "\n".join(latex)


def create_latex_table(df, caption, label, feature_set_name, use_longtable=True):
    """
    Create LaTeX table code from results dataframe.
    
    Args:
        df: Results dataframe
        caption: Table caption
        label: Table label for referencing
        feature_set_name: Name of feature set
        use_longtable: If True, use longtable (for multi-page tables), 
                      otherwise use table* (for single-page tables)
    
    Table fits full page width with columns:
    Feature | Low | Moderate | High | L vs M | L vs H | M vs H
    """
    if use_longtable or len(df) > 20:  # Use longtable for tables with >20 rows
        return create_latex_longtable(df, caption, label, feature_set_name)
    
    # Standard table for short tables
    latex = []
    
    # Table header
    latex.append("\\begin{table*}[ht]")
    latex.append("\\centering")
    latex.append("\\caption{" + caption + "}")
    latex.append("\\label{" + label + "}")
    latex.append("\\begin{tabular}{l c c c c c c}")
    latex.append("\\hline")
    latex.append("Feature & Low & Moderate & High & \\textit{p} L vs M & \\textit{p} L vs H & \\textit{p} M vs H \\\\")
    latex.append("\\hline")
    
    # Table rows
    for _, row in df.iterrows():
        feature_name = row['Feature'].replace('_', '\\_')  # Escape underscores
        
        # Format mean ± SD
        low_str = f"{row['Low_Mean']:.2f} $\\pm$ {row['Low_SD']:.2f}" if not pd.isna(row['Low_Mean']) else '-'
        mod_str = f"{row['Mod_Mean']:.2f} $\\pm$ {row['Mod_SD']:.2f}" if not pd.isna(row['Mod_Mean']) else '-'
        high_str = f"{row['High_Mean']:.2f} $\\pm$ {row['High_SD']:.2f}" if not pd.isna(row['High_Mean']) else '-'
        
        # Format p-values
        p_lm = format_pvalue(row['p_L_vs_M'])
        p_lh = format_pvalue(row['p_L_vs_H'])
        p_mh = format_pvalue(row['p_M_vs_H'])
        
        latex.append(f"{feature_name} & {low_str} & {mod_str} & {high_str} & {p_lm} & {p_lh} & {p_mh} \\\\")
    
    # Table footer
    latex.append("\\hline")
    latex.append("\\end{tabular}")
    latex.append("\\begin{tablenotes}")
    latex.append("\\small")
    latex.append("\\item Note: Values shown as Mean $\\pm$ SD. Significance: * $p < 0.05$, ** $p < 0.01$, *** $p < 0.001$.")
    latex.append("\\end{tablenotes}")
    latex.append("\\end{table*}")
    
    return "\n".join(latex)


print("✓ Formatting functions defined")

✓ Formatting functions defined


## Run Analysis for All Feature Sets

This will analyze all features and generate tables. Note: This may take several minutes to complete.

In [7]:
# Analyze Performance features
print("\n" + "="*60)
print("PERFORMANCE FEATURES")
print("="*60)
performance_results = analyze_all_features(performance_df, performance_features, "Performance")

R callback write-console: Loading required package: lme4
  
R callback write-console: Loading required package: Matrix
  



PERFORMANCE FEATURES

Analyzing Performance features

  Processing: sysmon_failure_rate
Means for sysmon_failure_rate:
  H: 11.424
  L: 6.182
  M: 6.645


R callback write-console: 
Attaching package: 'lmerTest'

  
R callback write-console: The following object is masked from 'package:lme4':

    lmer

  
R callback write-console: The following object is masked from 'package:stats':

    step

  
R callback write-console: Welcome to emmeans.
Caution: You lose important information if you filter this package's results.
See '? untidy'
  
R callback write-console: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient
  



=== sysmon_failure_rate (R lmerTest) ===
Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dv ~ condition + session_order_numeric + window_index + (1 |  
    participant_id)
   Data: dat

REML criterion at convergence: 15300.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.6035 -0.5808 -0.2381  0.2110  5.2611 

Random effects:
 Groups         Name        Variance Std.Dev.
 participant_id (Intercept)  41.5     6.442  
 Residual                   150.4    12.264  
Number of obs: 1935, groups:  participant_id, 43

Fixed effects:
              Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)  6.795e+00  1.117e+00 5.999e+01   6.086 8.84e-08 ***
condition.L  3.707e+00  4.829e-01 1.889e+03   7.677 2.60e-14 ***
condition.Q  1.762e+00  4.829e-01 1.889e+03   3.649 0.000271 ***
window_index 1.841e-01  6.453e-02 1.889e+03   2.853 0.004382 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Correlation 

R callback write-console: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient
  


...

    ✓ Complete

  Processing: resman_failure_rate
Means for resman_failure_rate:
  H: 82.013
  L: 27.866
  M: 46.220

=== resman_failure_rate (R lmerTest) ===
Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dv ~ condition + session_order_numeric + window_index + (1 |  
    participant_id)
   Data: dat

REML criterion at convergence: 18307.2

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-3.07925 -0.78375 -0.02044  0.76573  2.63537 

Random effects:
 Groups         Name        Variance Std.Dev.
 participant_id (Intercept) 711.2    26.67   
 Residual                   694.4    26.35   
Number of obs: 1935, groups:  participant_id, 43

Fixed effects:
              Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)    49.9013     4.2237   46.8098  11.815 1.20e-15 ***
condition.L    38.2879     1.0376 1889.0000  36.902  < 2e-16 ***
condition.Q     7.1196     1.0376 1889.0000   6.862 9.18e-12 ***
window_index    0.

R callback write-console: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient
  


    ✓ Complete

  Processing: average_accuracy
Means for average_accuracy:
  H: 59.713
  L: 83.455
  M: 77.142

=== average_accuracy (R lmerTest) ===
Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dv ~ condition + session_order_numeric + window_index + (1 |  
    participant_id)
   Data: dat

REML criterion at convergence: 14196.4

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.4719 -0.6490  0.0526  0.6822  2.7093 

Random effects:
 Groups         Name        Variance Std.Dev.
 participant_id (Intercept) 118.8    10.898  
 Residual                    82.0     9.055  
Number of obs: 1935, groups:  participant_id, 43

Fixed effects:
               Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)    75.31780    1.70755   45.39632   44.11  < 2e-16 ***
condition.L   -16.78768    0.35655 1889.00000  -47.08  < 2e-16 ***
condition.Q    -4.53842    0.35655 1889.00000  -12.73  < 2e-16 ***
window_index   -0.26874    0.

R callback write-console: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient
  


    ✓ Complete

  Processing: average_reaction_time
Means for average_reaction_time:
  H: 2.802
  L: 2.234
  M: 2.941

=== average_reaction_time (R lmerTest) ===
Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dv ~ condition + session_order_numeric + window_index + (1 |  
    participant_id)
   Data: dat

REML criterion at convergence: 8633.7

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.3911 -0.7739 -0.3352  0.6396  4.2809 

Random effects:
 Groups         Name        Variance Std.Dev.
 participant_id (Intercept) 0.1231   0.3509  
 Residual                   4.9572   2.2265  
Number of obs: 1935, groups:  participant_id, 43

Fixed effects:
               Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)   2.712e+00  1.102e-01  2.037e+02  24.601  < 2e-16 ***
condition.L   4.018e-01  8.767e-02  1.889e+03   4.583 4.89e-06 ***
condition.Q  -3.456e-01  8.767e-02  1.889e+03  -3.942 8.37e-05 ***
window_index -7.52

In [8]:
# Analyze Eye-tracking features
print("\n" + "="*60)
print("EYE-TRACKING FEATURES")
print("="*60)
eyetracking_results = analyze_all_features(eyetracking_df, eyetracking_features, "Eye-Tracking")

R callback write-console: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient
  



EYE-TRACKING FEATURES

Analyzing Eye-Tracking features

  Processing: pct_missing
Means for pct_missing:
  H: 8.434
  L: 7.636
  M: 8.591

=== pct_missing (R lmerTest) ===
Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dv ~ condition + session_order_numeric + window_index + (1 |  
    participant_id)
   Data: dat

REML criterion at convergence: 10001.4

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.8033 -0.5996 -0.1170  0.4377  5.5642 

Random effects:
 Groups         Name        Variance Std.Dev.
 participant_id (Intercept) 21.56    4.644   
 Residual                   10.92    3.305   
Number of obs: 1875, groups:  participant_id, 43

Fixed effects:
               Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)     9.11103    0.72308   44.60720  12.600 2.69e-16 ***
condition.L     0.53887    0.13279 1829.47342   4.058 5.16e-05 ***
condition.Q    -0.44263    0.13255 1829.33521  -3.339 0.000857 ***
window

R callback write-console: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient
  


    ✓ Complete

  Processing: mean_vel
Means for mean_vel:
  H: 1.655
  L: 1.530
  M: 1.606

=== mean_vel (R lmerTest) ===
Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dv ~ condition + session_order_numeric + window_index + (1 |  
    participant_id)
   Data: dat

REML criterion at convergence: 510.3

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.5186 -0.4927 -0.0804  0.3353 10.4484 

Random effects:
 Groups         Name        Variance Std.Dev.
 participant_id (Intercept) 0.11086  0.3330  
 Residual                   0.06875  0.2622  
Number of obs: 1875, groups:  participant_id, 43

Fixed effects:
               Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)   1.548e+00  5.208e-02  4.519e+01  29.723  < 2e-16 ***
condition.L   9.265e-02  1.053e-02  1.830e+03   8.795  < 2e-16 ***
condition.Q  -1.799e-02  1.052e-02  1.829e+03  -1.711   0.0873 .  
window_index  7.629e-03  1.409e-03  1.829e+03   5.416  6.9

R callback write-console: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient
  


...

    ✓ Complete

  Processing: sac_count
Means for sac_count:
  H: 3054.195
  L: 2967.002
  M: 2988.896

=== sac_count (R lmerTest) ===
Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dv ~ condition + session_order_numeric + window_index + (1 |  
    participant_id)
   Data: dat

REML criterion at convergence: 27094.3

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.6040 -0.5919 -0.0294  0.5559  4.5484 

Random effects:
 Groups         Name        Variance Std.Dev.
 participant_id (Intercept) 398409   631.2   
 Residual                    99804   315.9   
Number of obs: 1875, groups:  participant_id, 43

Fixed effects:
             Estimate Std. Error       df t value Pr(>|t|)    
(Intercept)  2886.968     97.266   43.273  29.681  < 2e-16 ***
condition.L    86.595     12.693 1829.223   6.822 1.22e-11 ***
condition.Q    -7.269     12.670 1829.153  -0.574    0.566    
window_index   16.123      1.697 1829.006   9.499  < 2e-16 *

R callback write-console: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient
  


    ✓ Complete

  Processing: sac_mean_dur
Means for sac_mean_dur:
  H: 0.002
  L: 0.002
  M: 0.002

=== sac_mean_dur (R lmerTest) ===
Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dv ~ condition + session_order_numeric + window_index + (1 |  
    participant_id)
   Data: dat

REML criterion at convergence: -34262.3

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.0974 -0.6083  0.0030  0.5835  3.9958 

Random effects:
 Groups         Name        Variance  Std.Dev. 
 participant_id (Intercept) 2.003e-09 4.475e-05
 Residual                   5.733e-10 2.394e-05
Number of obs: 1875, groups:  participant_id, 43

Fixed effects:
               Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)   2.362e-03  6.906e-06  4.339e+01 341.972  < 2e-16 ***
condition.L   7.916e-06  9.621e-07  1.829e+03   8.228 3.56e-16 ***
condition.Q   1.389e-06  9.603e-07  1.829e+03   1.447   0.1481    
window_index -2.568e-07  1.286e-07  1

R callback write-console: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient
  


    ✓ Complete

  Processing: sac_rate
Means for sac_rate:
  H: 50.904
  L: 49.451
  M: 49.816

=== sac_rate (R lmerTest) ===
Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dv ~ condition + session_order_numeric + window_index + (1 |  
    participant_id)
   Data: dat

REML criterion at convergence: 11773.4

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.6040 -0.5920 -0.0294  0.5560  4.5484 

Random effects:
 Groups         Name        Variance Std.Dev.
 participant_id (Intercept) 110.67   10.520  
 Residual                    27.72    5.265  
Number of obs: 1875, groups:  participant_id, 43

Fixed effects:
               Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)    48.11689    1.62113   43.27267  29.681  < 2e-16 ***
condition.L     1.44328    0.21156 1829.22279   6.822 1.22e-11 ***
condition.Q    -0.12114    0.21118 1829.15335  -0.574    0.566    
window_index    0.26874    0.02829 1829.00605   9.499

In [9]:
# Analyze GSR features
print("\n" + "="*60)
print("GSR FEATURES")
print("="*60)
gsr_results = analyze_all_features(gsr_df, gsr_features, "GSR")

R callback write-console: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient
  



GSR FEATURES

Analyzing GSR features

  Processing: SCR_Peaks_N
Means for SCR_Peaks_N:
  H: 7.291
  L: 7.338
  M: 7.327

=== SCR_Peaks_N (R lmerTest) ===
Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dv ~ condition + session_order_numeric + window_index + (1 |  
    participant_id)
   Data: dat

REML criterion at convergence: 7825.8

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.0806 -0.6750 -0.0779  0.6228  4.0601 

Random effects:
 Groups         Name        Variance Std.Dev.
 participant_id (Intercept) 4.222    2.055   
 Residual                   4.319    2.078   
Number of obs: 1782, groups:  participant_id, 40

Fixed effects:
               Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)   7.254e+00  3.381e-01  4.370e+01  21.458   <2e-16 ***
condition.L  -4.269e-02  8.545e-02  1.739e+03  -0.500    0.617    
condition.Q  -4.586e-03  8.525e-02  1.739e+03  -0.054    0.957    
window_index  8.349e-03  

R callback write-console: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient
  


    ✓ Complete

  Processing: SCR_Peaks_Amplitude_Mean
Means for SCR_Peaks_Amplitude_Mean:
  H: 0.286
  L: 0.214
  M: 0.309

=== SCR_Peaks_Amplitude_Mean (R lmerTest) ===
Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dv ~ condition + session_order_numeric + window_index + (1 |  
    participant_id)
   Data: dat

REML criterion at convergence: -2081.3

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.1469 -0.4564 -0.0825  0.4133  7.7405 

Random effects:
 Groups         Name        Variance Std.Dev.
 participant_id (Intercept) 0.06100  0.2470  
 Residual                   0.01595  0.1263  
Number of obs: 1782, groups:  participant_id, 40

Fixed effects:
               Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)   2.529e-01  3.946e-02  4.019e+01   6.409 1.23e-07 ***
condition.L   5.145e-02  5.192e-03  1.739e+03   9.910  < 2e-16 ***
condition.Q  -4.889e-02  5.180e-03  1.739e+03  -9.438  < 2e-16 ***
window_i

R callback write-console: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient
  


    ✓ Complete

  Processing: EDA_Tonic_SD
Means for EDA_Tonic_SD:
  H: 0.174
  L: 0.151
  M: 0.183

=== EDA_Tonic_SD (R lmerTest) ===
Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dv ~ condition + session_order_numeric + window_index + (1 |  
    participant_id)
   Data: dat

REML criterion at convergence: -3390.9

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.7936 -0.5167 -0.1162  0.3342  5.9622 

Random effects:
 Groups         Name        Variance Std.Dev.
 participant_id (Intercept) 0.01180  0.10860 
 Residual                   0.00779  0.08826 
Number of obs: 1782, groups:  participant_id, 40

Fixed effects:
               Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)   1.566e-01  1.762e-02  4.202e+01   8.886 3.35e-11 ***
condition.L   1.613e-02  3.629e-03  1.739e+03   4.446 9.30e-06 ***
condition.Q  -1.753e-02  3.621e-03  1.739e+03  -4.841 1.41e-06 ***
window_index  1.773e-03  4.847e-04  1.739e+0

R callback write-console: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient
  


    ✓ Complete

  Processing: EDA_Autocorrelation
Means for EDA_Autocorrelation:
  H: 0.351
  L: 0.372
  M: 0.347

=== EDA_Autocorrelation (R lmerTest) ===
Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dv ~ condition + session_order_numeric + window_index + (1 |  
    participant_id)
   Data: dat

REML criterion at convergence: -570.1

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.1087 -0.6606  0.0429  0.6914  2.7196 

Random effects:
 Groups         Name        Variance Std.Dev.
 participant_id (Intercept) 0.02444  0.1563  
 Residual                   0.03879  0.1969  
Number of obs: 1782, groups:  participant_id, 40

Fixed effects:
               Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)   3.450e-01  2.626e-02  4.631e+01  13.141   <2e-16 ***
condition.L  -1.518e-02  8.097e-03  1.739e+03  -1.875   0.0609 .  
condition.Q   1.250e-02  8.079e-03  1.739e+03   1.547   0.1221    
window_index  1.756e-03 

In [10]:
# Analyze ECG features
print("\n" + "="*60)
print("ECG FEATURES")
print("="*60)
ecg_results = analyze_all_features(ecg_df, ecg_features, "ECG")

R callback write-console: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient
  



ECG FEATURES

Analyzing ECG features

  Processing: HRV_MeanNN
Means for HRV_MeanNN:
  H: 738.376
  L: 699.796
  M: 712.526

=== HRV_MeanNN (R lmerTest) ===
Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dv ~ condition + session_order_numeric + window_index + (1 |  
    participant_id)
   Data: dat

REML criterion at convergence: 19094.1

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.5032 -0.2231 -0.0097  0.2007 11.6180 

Random effects:
 Groups         Name        Variance Std.Dev.
 participant_id (Intercept) 14827    121.8   
 Residual                    4569     67.6   
Number of obs: 1680, groups:  participant_id, 38

Fixed effects:
              Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)  7.254e+02  2.000e+01 3.794e+01  36.266  < 2e-16 ***
condition.L  1.816e+01  2.869e+00 1.639e+03   6.330 3.17e-10 ***
condition.Q  8.965e-02  2.869e+00 1.639e+03   0.031    0.975    
window_index 8.219e-03  3.817

R callback write-console: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient
  


    ✓ Complete

  Processing: HRV_SDNN
Means for HRV_SDNN:
  H: 138.388
  L: 94.567
  M: 110.967

=== HRV_SDNN (R lmerTest) ===
Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dv ~ condition + session_order_numeric + window_index + (1 |  
    participant_id)
   Data: dat

REML criterion at convergence: 19711.1

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.9858 -0.1341 -0.0321  0.1081 12.4401 

Random effects:
 Groups         Name        Variance Std.Dev.
 participant_id (Intercept) 51847    227.70  
 Residual                    6473     80.45  
Number of obs: 1680, groups:  participant_id, 38

Fixed effects:
              Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)   126.1835    37.1280   37.3517   3.399 0.001623 ** 
condition.L    12.4705     3.4149 1639.1599   3.652 0.000269 ***
condition.Q    -6.1907     3.4150 1638.9232  -1.813 0.070048 .  
window_index    0.8448     0.4543 1638.8047   1.859 0.06314

R callback write-console: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient
  


...

    ✓ Complete

  Processing: HRV_KFD
Means for HRV_KFD:
  H: 2.888
  L: 2.862
  M: 2.841

=== HRV_KFD (R lmerTest) ===
Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dv ~ condition + session_order_numeric + window_index + (1 |  
    participant_id)
   Data: dat

REML criterion at convergence: 3220.8

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.5699 -0.5970 -0.1048  0.4738  5.2803 

Random effects:
 Groups         Name        Variance Std.Dev.
 participant_id (Intercept) 0.2252   0.4745  
 Residual                   0.3650   0.6042  
Number of obs: 1680, groups:  participant_id, 38

Fixed effects:
               Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)   2.963e+00  8.198e-02  4.426e+01  36.143  < 2e-16 ***
condition.L   2.334e-02  2.563e-02  1.643e+03   0.911    0.363    
condition.Q   3.136e-02  2.564e-02  1.640e+03   1.223    0.221    
window_index -1.488e-02  3.412e-03  1.639e+03  -4.361 1.38e-

R callback write-console: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient
  


    ✓ Complete

  Processing: HRV_LZC
Means for HRV_LZC:
  H: 0.970
  L: 0.955
  M: 0.972

=== HRV_LZC (R lmerTest) ===
Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dv ~ condition + session_order_numeric + window_index + (1 |  
    participant_id)
   Data: dat

REML criterion at convergence: -1769.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.3755 -0.6207  0.0659  0.6890  3.0265 

Random effects:
 Groups         Name        Variance Std.Dev.
 participant_id (Intercept) 0.01356  0.1165  
 Residual                   0.01852  0.1361  
Number of obs: 1680, groups:  participant_id, 38

Fixed effects:
               Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)   9.933e-01  1.993e-02  4.293e+01  49.836  < 2e-16 ***
condition.L   8.149e-03  5.773e-03  1.642e+03   1.412   0.1582    
condition.Q  -9.565e-03  5.775e-03  1.640e+03  -1.656   0.0979 .  
window_index -3.559e-03  7.684e-04  1.639e+03  -4.631 3.92e

R callback write-console: Error: grouping factors must have > 1 sampled level
  
R callback write-console: Error: grouping factors must have > 1 sampled level
  
R callback write-console: Error: grouping factors must have > 1 sampled level
  
R callback write-console: Error: grouping factors must have > 1 sampled level
  
R callback write-console: Error: grouping factors must have > 1 sampled level
  
R callback write-console: Error: grouping factors must have > 1 sampled level
  
R callback write-console: Error: grouping factors must have > 1 sampled level
  
R callback write-console: Error: grouping factors must have > 1 sampled level
  
R callback write-console: Error: grouping factors must have > 1 sampled level
  


    ✓ Complete

  Processing: HRV_DFA_alpha2
Means for HRV_DFA_alpha2:
  H: 0.100
  L: nan
  M: 0.140
    ✗ Error: Error: grouping factors must have > 1 sampled level


  Processing: HRV_MFDFA_alpha2_Width
Means for HRV_MFDFA_alpha2_Width:
  H: 0.548
  L: nan
  M: 0.307
    ✗ Error: Error: grouping factors must have > 1 sampled level


  Processing: HRV_MFDFA_alpha2_Peak
Means for HRV_MFDFA_alpha2_Peak:
  H: 0.160
  L: nan
  M: 0.015
    ✗ Error: Error: grouping factors must have > 1 sampled level


  Processing: HRV_MFDFA_alpha2_Mean
Means for HRV_MFDFA_alpha2_Mean:
  H: 0.237
  L: nan
  M: 0.102
    ✗ Error: Error: grouping factors must have > 1 sampled level


  Processing: HRV_MFDFA_alpha2_Max
Means for HRV_MFDFA_alpha2_Max:
  H: -0.074
  L: nan
  M: 0.372
    ✗ Error: Error: grouping factors must have > 1 sampled level


  Processing: HRV_MFDFA_alpha2_Delta
Means for HRV_MFDFA_alpha2_Delta:
  H: -0.192
  L: nan
  M: -0.149
    ✗ Error: Error: grouping factors must have > 1 sampled

In [11]:
# Analyze Pose features
print("\n" + "="*60)
print("POSE FEATURES")
print("="*60)
pose_results = analyze_all_features(pose_df, pose_features, "Pose")

R callback write-console: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient
  



POSE FEATURES

Analyzing Pose features

  Processing: head_rotation_rad_rms
Means for head_rotation_rad_rms:
  H: 0.064
  L: 0.061
  M: 0.068

=== head_rotation_rad_rms (R lmerTest) ===
Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dv ~ condition + session_order_numeric + window_index + (1 |  
    participant_id)
   Data: dat

REML criterion at convergence: -9068.1

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.2279 -0.3954 -0.0628  0.3325 14.4795 

Random effects:
 Groups         Name        Variance  Std.Dev.
 participant_id (Intercept) 0.0012570 0.03545 
 Residual                   0.0005819 0.02412 
Number of obs: 2021, groups:  participant_id, 45

Fixed effects:
               Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)   6.157e-02  5.383e-03  4.639e+01  11.439 4.24e-15 ***
condition.L   1.788e-03  9.300e-04  1.973e+03   1.923  0.05468 .  
condition.Q  -4.933e-03  9.290e-04  1.973e+03  -5.310 1.

R callback write-console: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient
  


    ✓ Complete

  Processing: head_rotation_rad_mean
Means for head_rotation_rad_mean:
  H: 0.002
  L: -0.002
  M: 0.001

=== head_rotation_rad_mean (R lmerTest) ===
Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dv ~ condition + session_order_numeric + window_index + (1 |  
    participant_id)
   Data: dat

REML criterion at convergence: -9254.5

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-9.7363 -0.5079 -0.0107  0.4668  6.8544 

Random effects:
 Groups         Name        Variance Std.Dev.
 participant_id (Intercept) 0.002597 0.05096 
 Residual                   0.000521 0.02283 
Number of obs: 2021, groups:  participant_id, 45

Fixed effects:
               Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)   2.608e-03  7.658e-03  4.504e+01   0.341 0.734992    
condition.L   3.044e-03  8.800e-04  1.973e+03   3.459 0.000554 ***
condition.Q  -2.913e-04  8.790e-04  1.973e+03  -0.331 0.740433    
window_index 

R callback write-console: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient
  


...

    ✓ Complete

  Processing: pupil_metric_acc_rms
Means for pupil_metric_acc_rms:
  H: 3.454
  L: 3.540
  M: 3.091

=== pupil_metric_acc_rms (R lmerTest) ===
Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dv ~ condition + session_order_numeric + window_index + (1 |  
    participant_id)
   Data: dat

REML criterion at convergence: 7693.3

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.9352 -0.1963 -0.0461  0.1204 16.5565 

Random effects:
 Groups         Name        Variance Std.Dev.
 participant_id (Intercept) 1.793    1.339   
 Residual                   2.434    1.560   
Number of obs: 2018, groups:  participant_id, 45

Fixed effects:
               Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)   3.370e+00  2.103e-01  5.099e+01  16.024  < 2e-16 ***
condition.L  -7.402e-02  6.022e-02  1.970e+03  -1.229    0.219    
condition.Q   3.366e-01  6.010e-02  1.970e+03   5.601 2.43e-08 ***
window_index -5.734e-

R callback write-console: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient
  


    ✓ Complete

  Processing: pupil_metric_acc_mean
Means for pupil_metric_acc_mean:
  H: -0.005
  L: 0.002
  M: 0.002

=== pupil_metric_acc_mean (R lmerTest) ===
Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dv ~ condition + session_order_numeric + window_index + (1 |  
    participant_id)
   Data: dat

REML criterion at convergence: -6386.8

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-30.7775  -0.0554   0.0017   0.0963   2.3788 

Random effects:
 Groups         Name        Variance  Std.Dev.
 participant_id (Intercept) 0.0001714 0.01309 
 Residual                   0.0023430 0.04840 
Number of obs: 2018, groups:  participant_id, 45

Fixed effects:
               Estimate Std. Error         df t value Pr(>|t|)   
(Intercept)  -3.311e-03  2.831e-03  1.115e+02  -1.170  0.24462   
condition.L  -4.917e-03  1.868e-03  1.970e+03  -2.632  0.00855 **
condition.Q  -3.033e-03  1.865e-03  1.969e+03  -1.626  0.10403   
window_

R callback write-console: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient
  


    ✓ Complete

  Processing: pupil_metric_acc_std
Means for pupil_metric_acc_std:
  H: 3.454
  L: 3.540
  M: 3.091

=== pupil_metric_acc_std (R lmerTest) ===
Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: dv ~ condition + session_order_numeric + window_index + (1 |  
    participant_id)
   Data: dat

REML criterion at convergence: 7691.9

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.9366 -0.1963 -0.0461  0.1204 16.5407 

Random effects:
 Groups         Name        Variance Std.Dev.
 participant_id (Intercept) 1.793    1.339   
 Residual                   2.432    1.560   
Number of obs: 2018, groups:  participant_id, 45

Fixed effects:
               Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)   3.370e+00  2.103e-01  5.099e+01  16.026  < 2e-16 ***
condition.L  -7.409e-02  6.019e-02  1.970e+03  -1.231    0.219    
condition.Q   3.366e-01  6.008e-02  1.970e+03   5.602 2.42e-08 ***
window_index -5.639e-

## Save Results to CSV

In [12]:
# Save CSV files
performance_results.to_csv(OUTPUT_DIR / 'tables' / 'performance_statistics.csv', index=False)
print(f"✓ Saved: {OUTPUT_DIR / 'tables' / 'performance_statistics.csv'}")

eyetracking_results.to_csv(OUTPUT_DIR / 'tables' / 'eyetracking_statistics.csv', index=False)
print(f"✓ Saved: {OUTPUT_DIR / 'tables' / 'eyetracking_statistics.csv'}")

gsr_results.to_csv(OUTPUT_DIR / 'tables' / 'gsr_statistics.csv', index=False)
print(f"✓ Saved: {OUTPUT_DIR / 'tables' / 'gsr_statistics.csv'}")

ecg_results.to_csv(OUTPUT_DIR / 'tables' / 'ecg_statistics.csv', index=False)
print(f"✓ Saved: {OUTPUT_DIR / 'tables' / 'ecg_statistics.csv'}")

pose_results.to_csv(OUTPUT_DIR / 'tables' / 'pose_statistics.csv', index=False)
print(f"✓ Saved: {OUTPUT_DIR / 'tables' / 'pose_statistics.csv'}")

print("\n✓ All CSV files saved successfully")

✓ Saved: D:\VScode folders\MATB-Workload-Classification-Facial-Pose-Data\stats_utils\tables\performance_statistics.csv
✓ Saved: D:\VScode folders\MATB-Workload-Classification-Facial-Pose-Data\stats_utils\tables\eyetracking_statistics.csv
✓ Saved: D:\VScode folders\MATB-Workload-Classification-Facial-Pose-Data\stats_utils\tables\gsr_statistics.csv
✓ Saved: D:\VScode folders\MATB-Workload-Classification-Facial-Pose-Data\stats_utils\tables\ecg_statistics.csv
✓ Saved: D:\VScode folders\MATB-Workload-Classification-Facial-Pose-Data\stats_utils\tables\pose_statistics.csv

✓ All CSV files saved successfully


## Generate LaTeX Tables

In [13]:
# Generate LaTeX tables
performance_latex = create_latex_table(
    performance_results,
    caption="Statistical analysis of task performance features across workload conditions.",
    label="tab:performance_stats",
    feature_set_name="Performance"
)

eyetracking_latex = create_latex_table(
    eyetracking_results,
    caption="Statistical analysis of eye-tracking features across workload conditions.",
    label="tab:eyetracking_stats",
    feature_set_name="Eye-Tracking"
)

gsr_latex = create_latex_table(
    gsr_results,
    caption="Statistical analysis of galvanic skin response features across workload conditions.",
    label="tab:gsr_stats",
    feature_set_name="GSR"
)

ecg_latex = create_latex_table(
    ecg_results,
    caption="Statistical analysis of heart rate variability features across workload conditions.",
    label="tab:ecg_stats",
    feature_set_name="ECG"
)

pose_latex = create_latex_table(
    pose_results,
    caption="Statistical analysis of facial pose features across workload conditions.",
    label="tab:pose_stats",
    feature_set_name="Pose"
)

# Save LaTeX files
with open(OUTPUT_DIR / 'tables' / 'performance_table.tex', 'w') as f:
    f.write(performance_latex)
print(f"✓ Saved: {OUTPUT_DIR / 'tables' / 'performance_table.tex'}")

with open(OUTPUT_DIR / 'tables' / 'eyetracking_table.tex', 'w') as f:
    f.write(eyetracking_latex)
print(f"✓ Saved: {OUTPUT_DIR / 'tables' / 'eyetracking_table.tex'}")

with open(OUTPUT_DIR / 'tables' / 'gsr_table.tex', 'w') as f:
    f.write(gsr_latex)
print(f"✓ Saved: {OUTPUT_DIR / 'tables' / 'gsr_table.tex'}")

with open(OUTPUT_DIR / 'tables' / 'ecg_table.tex', 'w') as f:
    f.write(ecg_latex)
print(f"✓ Saved: {OUTPUT_DIR / 'tables' / 'ecg_table.tex'}")

with open(OUTPUT_DIR / 'tables' / 'pose_table.tex', 'w') as f:
    f.write(pose_latex)
print(f"✓ Saved: {OUTPUT_DIR / 'tables' / 'pose_table.tex'}")

print("\n✓ All LaTeX files saved successfully")

✓ Saved: D:\VScode folders\MATB-Workload-Classification-Facial-Pose-Data\stats_utils\tables\performance_table.tex
✓ Saved: D:\VScode folders\MATB-Workload-Classification-Facial-Pose-Data\stats_utils\tables\eyetracking_table.tex
✓ Saved: D:\VScode folders\MATB-Workload-Classification-Facial-Pose-Data\stats_utils\tables\gsr_table.tex
✓ Saved: D:\VScode folders\MATB-Workload-Classification-Facial-Pose-Data\stats_utils\tables\ecg_table.tex
✓ Saved: D:\VScode folders\MATB-Workload-Classification-Facial-Pose-Data\stats_utils\tables\pose_table.tex

✓ All LaTeX files saved successfully


## Display Sample LaTeX Code

Here's a sample of the Performance table LaTeX code for copy-paste into Overleaf:

In [14]:
print("\n" + "="*80)
print("SAMPLE LATEX CODE (Performance Table)")
print("="*80)
print(performance_latex)
print("\n" + "="*80)
print("\nNote: Add this to your LaTeX document preamble:")
print("\\usepackage{threeparttable}")
print("="*80)


SAMPLE LATEX CODE (Performance Table)
\begin{longtable}{l c c c c c c}
\caption{Statistical analysis of task performance features across workload conditions.} \label{tab:performance_stats} \\
\hline
Feature & Low & Moderate & High & \textit{p} L vs M & \textit{p} L vs H & \textit{p} M vs H \\
\hline
\endfirsthead

\multicolumn{7}{c}
{\tablename\ \thetable\ -- \textit{Continued from previous page}} \\
\hline
Feature & Low & Moderate & High & \textit{p} L vs M & \textit{p} L vs H & \textit{p} M vs H \\
\hline
\endhead

\hline \multicolumn{7}{r}{\textit{Continued on next page}} \\
\endfoot

\hline
\multicolumn{7}{l}{\textit{Note: Values shown as Mean $\pm$ SD. Significance: * $p < 0.05$, ** $p < 0.01$, *** $p < 0.001$.}} \\
\endlastfoot

sysmon\_failure\_rate & 6.18 $\pm$ 13.57 & 6.65 $\pm$ 13.48 & 11.42 $\pm$ 14.46 & 0.776 & <0.001*** & <0.001*** \\
sysmon\_average\_reaction\_times & 1434.02 $\pm$ 2082.28 & 2941.90 $\pm$ 1535.57 & 3256.16 $\pm$ 1549.11 & <0.001*** & <0.001*** & 0.003** 

## Summary

All statistical analyses are complete. Files generated:

**CSV Files:**
- `performance_statistics.csv`
- `eyetracking_statistics.csv`
- `gsr_statistics.csv`
- `ecg_statistics.csv`
- `pose_statistics.csv`

**LaTeX Files:**
- `performance_table.tex`
- `eyetracking_table.tex`
- `gsr_table.tex`
- `ecg_table.tex`
- `pose_table.tex`

## Using the LaTeX Tables in Overleaf

The generated tables use the `longtable` package, which allows tables to automatically span multiple pages. This is essential for large tables with many features.

**Required LaTeX packages (add to your preamble):**
```latex
\usepackage{longtable}
\usepackage{booktabs}
```

**Features of longtable:**
- Automatically breaks across pages
- Repeats header on each page
- Shows "Continued from previous page" and "Continued on next page" messages
- Places table notes at the end of the table

**To include a table in your document:**
Simply copy the entire contents of the `.tex` file into your LaTeX document where you want the table to appear. No need for `\begin{table}` or `\end{table}` - longtable handles everything.

**For shorter tables (<20 rows):**
The code automatically uses standard `table*` environment instead of `longtable`. These tables include the `threeparttable` package for notes, so also add:
```latex
\usepackage{threeparttable}
```